<a href="https://colab.research.google.com/github/gawankarsanket/classification_networkintrusion/blob/main/Capstone_Classification_NetSecML_BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NETWORK INTRUSION DETECTION MODEL

##We have been tasked to build and demonstrte ML model for network intrusion detection Objective is model should be able to determine whther the traffic passing the firewall is genuine traffic or maliciuos traffic.


##For the model  preparation, we recived the simulated/dummy dataset from customer, loaded in AWS S3.


##The given dataset consists of approximately 5 million records, each characterized by 41 features. These features encompass a wide range of attributes, including connection duration, protocol type, service, source and destination addresses, flags, and more. Additionally, each record is labeled with the corresponding attack type or labeled as normal traffic.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
 # BOTO3 is  AWS SDK for Python that provides a Python API for AWS infrastructure services.
!pip install boto3 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.4 MB/s eta 0:00:00


#Access AWS S3 and download file from there

In [ ]:
#import python SDK ie API for AWS services
import boto3


#Access your secret keys in Python via:
from google.colab import userdata

BUCKET_NAME =userdata.get('bucketName')
ACCESS_KEY= userdata.get('accessKey')
SECRET_KEY= userdata.get('secretKey')


# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key= SECRET_KEY)

#s3 is object created which is  authrosied using access keys & secret key to  access the S3 bucket in AWS

In [ ]:
# save the filename in KEY variable
KEY = 'netsecdata.gz' # replace with your object key

try:
  # we are trying to download file from s3 with name `netsecdata.gz` stored in varaible KEY
  # to colab dir with name `netsecdata.gz`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'netsecdata.gz')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

In [ ]:
#downloaded file is in gz format, we need to unzip it.
import gzip

with gzip.open('netsecdata.gz', 'rb') as f:
  df_rawfile= pd.read_csv(f,header=None)

In [ ]:
df_rawfile.head(10)

# there is no header. Need to figure out the header lables. So far we can see there are 42 columns

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
5,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.
6,0,tcp,http,SF,235,1337,0,0,0,0,...,6,1.0,0.0,0.17,0.0,0.0,0.0,0.0,0.0,normal.
7,0,tcp,http,SF,234,1364,0,0,0,0,...,7,1.0,0.0,0.14,0.0,0.0,0.0,0.0,0.0,normal.
8,0,tcp,http,SF,239,1295,0,0,0,0,...,8,1.0,0.0,0.12,0.0,0.0,0.0,0.0,0.0,normal.
9,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
df_rawfile.columns

#There are no column names, we will map columns names from file label.csv

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41],
      dtype='int64')

In [ ]:
#This is file has label/header informations

# save the filename in label variable
lables = 'lables.csv'

try:
  # we are trying to download file from s3 with name `lables.pdf` stored in varaible lables
  # to colab dir with name `lables.pdf`
  s3.Bucket(BUCKET_NAME).download_file(lables, 'lables.csv')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

In [ ]:
label_file=pd.read_csv("/content/lables.csv")
label_file

,Labels
0,duration
1,protocol_type
2,service
3,flag
4,src_bytes
5,dst_bytes
6,land
7,wrong_fragment
8,urgent
9,hot


In [ ]:
#converting this dataframe column to list
label_list=label_file['Labels'].to_list()

# list will be mapped to df.columns which will become its headers
df_rawfile.columns=label_list

df_rawfile.head(3)



,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
df_rawfile.info(),df_rawfile.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 42 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   duration                     int64  
 1   protocol_type                object 
 2   service                      object 
 3   flag                         object 
 4   src_bytes                    int64  
 5   dst_bytes                    int64  
 6   land                         int64  
 7   wrong_fragment               int64  
 8   urgent                       int64  
 9   hot                          int64  
 10  num_failed_logins            int64  
 11  logged_in                    int64  
 12  num_compromised              int64  
 13  root_shell                   int64  
 14  su_attempted                 int64  
 15  num_root                     int64  
 16  num_file_creations           int64  
 17  num_shells                   int64  
 18  num_access_files             int64  
 19  

(None, (4898431, 42))

## Number of rows =48lacs
## Number of cols = 42 (41 input features and 1 target feature (class))

## Given the scale of the demo model and the size of the dataset (48 lakh rows), training the model with the entire dataset would be time-consuming. Therefore, we'll be using a portion of the dataset for this demo purpose.

In [ ]:
df=df_rawfile.sample(frac=0.1,random_state=7)   # considering 10% of rows for this demo

df.shape

(489843, 42)

In [ ]:
#checkout Target variable
df['class'].value_counts()

#Here we have 23 diff categories. one of them is normal ie geniune class others are attacks.

# lets classify them as Normal= 0 and for rest attack = 1

class
smurf.              280821
neptune.            107136
normal.              97356
satan.                1561
ipsweep.              1252
portsweep.            1017
nmap.                  236
back.                  230
warezclient.           107
teardrop.               83
pod.                    31
guess_passwd.            3
land.                    3
loadmodule.              2
buffer_overflow.         2
warezmaster.             1
ftp_write.               1
rootkit.                 1
Name: count, dtype: int64

In [ ]:
attack=list(df['class'].unique())
attack

['smurf.',
 'normal.',
 'neptune.',
 'satan.',
 'back.',
 'portsweep.',
 'teardrop.',
 'ipsweep.',
 'nmap.',
 'land.',
 'pod.',
 'warezclient.',
 'buffer_overflow.',
 'loadmodule.',
 'guess_passwd.',
 'warezmaster.',
 'ftp_write.',
 'rootkit.']

In [ ]:
#remove 'normal' from this list

attack.pop(1)
attack

['smurf.',
 'neptune.',
 'satan.',
 'back.',
 'portsweep.',
 'teardrop.',
 'ipsweep.',
 'nmap.',
 'land.',
 'pod.',
 'warezclient.',
 'buffer_overflow.',
 'loadmodule.',
 'guess_passwd.',
 'warezmaster.',
 'ftp_write.',
 'rootkit.']

In [ ]:
#Mapping 0 and 1

class_mappping={'normal.':0,**{cls:1 for cls in attack}}
df['isMalicious']=df['class'].map(class_mappping)
df.drop(['class'],axis=1,inplace=True)
df.head(2)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,isMalicious
4045743,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
2184221,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
df.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,isMalicious
count,489843.000000,4.898430e+05,4.898430e+05,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,...,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000,489843.000000
mean,48.272226,1.302154e+03,1.478729e+03,0.000010,0.000568,0.000006,0.013133,0.000029,0.143732,0.005265,...,189.323061,0.754055,0.030691,0.605232,0.006521,0.177674,0.177421,0.058434,0.058146,0.801251
std,710.791277,3.112958e+05,5.723416e+05,0.003195,0.039644,0.003195,0.482805,0.006390,0.350818,1.412182,...,105.874402,0.411026,0.108481,0.480966,0.041579,0.381500,0.381778,0.231948,0.231861,0.399059
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,50.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,42616.000000,2.172773e+08,4.002911e+08,1.000000,3.000000,2.000000,44.000000,2.000000,1.000000,622.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#creating df1 as copy of df and procees it further

df1=df.copy()


#Convert object type data in numeric



In [ ]:
# Label encode Input feautres.  #Creating function so as to re-use with real test data

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

def labelencoding(df):
  for col in df.columns:
    if df[col].dtypes=='object':
      name='num'+col                       #creating heading for that column
      df[name]=le.fit_transform(df[col])   # Column with encoded lable is added
      df.drop(col,axis=1,inplace=True)    #  column with object type will be removed

  return df

"""
for col in df1.columns:
  if df1[col].dtypes=='object':
    name='num_'+col                        #creating heading for that column
    df1[name] = le.fit_transform(df1[col])
    df1.drop(col,axis=1,inplace=True)
"""

df1=labelencoding(df1)

df1.head(2)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,isMalicious,numprotocol_type,numservice,numflag
4045743,0,520,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1,0,15,9
2184221,0,1032,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1,0,15,9


In [ ]:
df1.info()  # Now all columns in numeric format

<class 'pandas.core.frame.DataFrame'>
Index: 489843 entries, 4045743 to 2810729
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     489843 non-null  int64  
 1   src_bytes                    489843 non-null  int64  
 2   dst_bytes                    489843 non-null  int64  
 3   land                         489843 non-null  int64  
 4   wrong_fragment               489843 non-null  int64  
 5   urgent                       489843 non-null  int64  
 6   hot                          489843 non-null  int64  
 7   num_failed_logins            489843 non-null  int64  
 8   logged_in                    489843 non-null  int64  
 9   num_compromised              489843 non-null  int64  
 10  root_shell                   489843 non-null  int64  
 11  su_attempted                 489843 non-null  int64  
 12  num_root                     489843 non-null  int64  
 1

#FEATURE SELECTION

### There are in total 41 input feature. Many of them may not be useful or affect output. We shall find them and remove.

## 1. Variance Threshold  - Low variance features will be removed.

### Some may be highly corelated with each other, we can use one of them instead of them all. (Corr)

### We can also apply RCF and get best 15 features to process the data ahead

##Variance Threshold

In [ ]:
df1.var()



duration                       5.052242e+05
src_bytes                      9.690506e+10
dst_bytes                      3.275749e+11
land                           1.020727e-05
wrong_fragment                 1.571613e-03
urgent                         1.020734e-05
hot                            2.331007e-01
num_failed_logins              4.082868e-05
logged_in                      1.230732e-01
num_compromised                1.994258e+00
root_shell                     6.940532e-05
su_attempted                   7.757402e-05
num_root                       2.466560e+00
num_file_creations             2.193770e-02
num_shells                     8.369345e-05
num_access_files               1.207617e-03
num_outbound_cmds              0.000000e+00
is_host_login                  0.000000e+00
is_guest_login                 8.607601e-04
count                          4.495997e+04
srv_count                      6.050663e+04
serror_rate                    1.455630e-01
srv_serror_rate                1

In [ ]:
#Split in X and Y

x=df1.drop('isMalicious',axis=1)
y=df1['isMalicious']

In [ ]:
# Applying VarianceThreshold and get the columns with variance 0.1 or less,  to be removed

from sklearn.feature_selection import VarianceThreshold

#create VarThreshold object
vt=VarianceThreshold(threshold=0.1)
vt.fit(x)

VarianceThreshold(threshold=0.1)

In [ ]:
vt.get_support()

array([ True,  True,  True, False, False, False,  True, False,  True,
        True, False, False,  True, False, False, False, False, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True,  True])

In [ ]:
# Get the columns which less that threshold,ie False

low_var_col=x.columns[vt.get_support()==False]
low_var_col

Index(['land', 'wrong_fragment', 'urgent', 'num_failed_logins', 'root_shell',
       'su_attempted', 'num_file_creations', 'num_shells', 'num_access_files',
       'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'rerror_rate',
       'srv_rerror_rate', 'diff_srv_rate', 'srv_diff_host_rate',
       'dst_host_diff_srv_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'],
      dtype='object')

In [ ]:
#Remove these low varivance columns

x.drop(low_var_col, axis=1,inplace=True)
x.shape

(489843, 21)

## Here we have reduced the columns of low variane. Now we will apply recursive feature elimination (RFE) and get best fit 15 columns only

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
## will use DT to figured best possible combination for 15 columns

DT=DecisionTreeClassifier()

selector = RFE(DT, n_features_to_select=15, step=1)
selector = selector.fit(x, y)





In [ ]:
selector

RFE(estimator=DecisionTreeClassifier(), n_features_to_select=15)

In [ ]:
# Post processing RFE, columns are marked if they qualify for selection as per given rule mut be in top 15 best fit columns
selector.support_

array([False,  True,  True,  True, False,  True, False,  True,  True,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
#Their ranking as per selection. The one with rank 1 are quaiified for selecton
selector.ranking_

array([2, 1, 1, 1, 3, 1, 6, 1, 1, 5, 7, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# find total number of feature and their names
len(selector.get_feature_names_out()), selector.get_feature_names_out()


(15,
 array(['src_bytes', 'dst_bytes', 'hot', 'num_compromised', 'count',
        'srv_count', 'dst_host_count', 'dst_host_srv_count',
        'dst_host_same_srv_rate', 'dst_host_same_src_port_rate',
        'dst_host_serror_rate', 'dst_host_srv_serror_rate',
        'numprotocol_type', 'numservice', 'numflag'], dtype=object))

In [ ]:
feature_selected=selector.get_feature_names_out()
x1=x[feature_selected]

x1.head(3)


,src_bytes,dst_bytes,hot,num_compromised,count,srv_count,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,numprotocol_type,numservice,numflag
4045743,520,0,0,0,481,481,255,255,1.0,1.00,0.0,0.0,0,15,9
2184221,1032,0,0,0,511,511,255,255,1.0,1.00,0.0,0.0,0,15,9
236495,303,2406,0,0,5,5,104,255,1.0,0.01,0.0,0.0,1,23,9


In [ ]:
#Saving this dataframe so as to generate sample input file for user.
x1.head().to_csv('input.csv', index=False)

## Here we get the datafrme with top 15 best fit columns,
## Now we should apply ML algorithms and train the models.
## However lets take a look at target class distribution and find if data is balanced of not.


In [ ]:
y.value_counts()

# Dataset is imbalance and this may lead to inaccurate training .

# We need to upsample the data to make dataset balanced. Will use SMOTE

isMalicious
1    392487
0     97356
Name: count, dtype: int64

#SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

p,q=SMOTE().fit_resample(x1,y)

In [ ]:
p.shape,q.shape,q.value_counts()

((784974, 15),
 (784974,),
 isMalicious
 1    392487
 0    392487
 Name: count, dtype: int64)

In [ ]:
type(p),type(q)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

#Now dataset is balanced. Lets perform train test split

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(p,q,test_size=0.3,random_state=7)

xtrain.shape,ytrain.shape, xtest.shape, ytest.shape

((549481, 15), (549481,), (235493, 15), (235493,))

##  We will use  Decision Tree, RandomForest and XGboost with default parameter and hyperparamter tuning.
## we will calculate the precision and recall to compare the performance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
## CODE FOR MODEL EVALUTION WHILE WITH HYPERPARAMTER TUNING

from sklearn.metrics import f1_score

def evaluate_model (model,xtrain,ytrain,xtest,ytest):
  report={}
  model.fit(xtrain, ytrain)
  ypred=model.predict(xtest)
  accuracy=accuracy_score(ytest,ypred)*100
  precision=precision_score(ytest,ypred)*100
  recall=recall_score(ytest,ypred)*100
  f1=f1_score(ytest,ypred)*100

  best_parameter=model.best_params_


  return ypred,accuracy,precision,recall,f1,best_parameter

# DECISION TREE with default parameter




In [ ]:
DT_model_dflt= DecisionTreeClassifier(random_state=7)
DT_model_dflt.fit(xtrain,ytrain)
ypred=DT_model_dflt.predict(xtest)
DT_dflt_accuracy = accuracy_score(ytest, ypred)*100
DT_dflt_precision = precision_score(ytest, ypred)*100
DT_dflt_recall = recall_score(ytest, ypred)*100
DT_dflt_f1 = f1_score(ytest, ypred)*100


print(f'Accuracy:{DT_dflt_accuracy} Precision:{DT_dflt_precision},Recall:{DT_dflt_recall},F1_score:{DT_dflt_f1}')

Accuracy:99.98598684461959 Precision:99.98895665913454,Recall:99.98301125504354,F1_score:99.98598386870707


#DECISION TREE with hypter paramter tuning with randomserch CV

In [ ]:

## Hypeparameter Tuning for DT
params_DT={'max_depth':[3,5,10,None],
           'criterion':['gini','entropy']
           }

DT_model_hpt=RandomizedSearchCV(DecisionTreeClassifier(),param_distributions=params_DT,scoring='f1',cv=2,verbose=2)

DT_hpt_ypred,DT_hpt_accuracy,DT_hpt_precison,DT_hpt_recall,DT_hpt_f1,DT_hpt_best_parameter=evaluate_model(DT_model_hpt,xtrain,ytrain,xtest,ytest)

print (f' Accuracy:{DT_hpt_accuracy} Precision:{DT_hpt_precison} Recall:{DT_hpt_recall} F1 score:{DT_hpt_f1},Best parameter:{DT_hpt_best_parameter}')


Fitting 2 folds for each of 8 candidates, totalling 16 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ........................criterion=gini, max_depth=3; total time=   0.6s
[CV] END ........................criterion=gini, max_depth=3; total time=   0.6s
[CV] END ........................criterion=gini, max_depth=5; total time=   0.8s
[CV] END ........................criterion=gini, max_depth=5; total time=   0.8s
[CV] END .......................criterion=gini, max_depth=10; total time=   1.1s
[CV] END .......................criterion=gini, max_depth=10; total time=   1.1s
[CV] END .....................criterion=gini, max_depth=None; total time=   1.3s
[CV] END .....................criterion=gini, max_depth=None; total time=   2.1s
[CV] END .....................criterion=entropy, max_depth=3; total time=   0.9s
[CV] END .....................criterion=entropy, max_depth=3; total time=   0.7s
[CV] END .....................criterion=entropy, max_depth=5; total time=   0.8s
[CV] END .....................criterion=entropy, max_depth=5; total time=   0.8s
[CV] END ...................

#Random Forest with Default Parameter

In [ ]:
## RF taking too much of time hence parking it aside for while


RF_model_dflt= RandomForestClassifier(random_state=7)
RF_model_dflt.fit(xtrain,ytrain)
ypred=RF_model_dflt.predict(xtest)
RF_dflt_accuracy = accuracy_score(ytest, ypred)*100
RF_dflt_precision = precision_score(ytest, ypred)*100
RF_dflt_recall = recall_score(ytest, ypred)*100
RF_dflt_f1 = f1_score(ytest, ypred)*100


print(f'Accuracy:{RF_dflt_accuracy} Precision:{RF_dflt_precision},Recall:{RF_dflt_recall},F1_score:{RF_dflt_f1}')

Accuracy:99.99108253748518 Precision:99.99915042563676,Recall:99.98301125504354,F1_score:99.99108018909999


#Random Forest with hypter paramter tuning with randomserch CV

In [ ]:
## RF taking too much of time hence parking it aside for while


## Hypeparameter Tuning for RandomForest
params_RF={'max_depth':[3,5,10,None],
        'n_estimators':[50,150,200],
        'criterion':['gini','entropy'],
        'oob_score':[True]
           }

RF_model_hpt=RandomizedSearchCV(RandomForestClassifier(),param_distributions=params_RF,scoring='f1',cv=2,verbose=2)


RF_hpt_ypred,RF_hpt_accuracy,RF_hpt_precison,RF_hpt_recall,RF_hpt_f1,RF_hpt_best_parameter=evaluate_model(RF_model_hpt,xtrain,ytrain,xtest,ytest)


print (f' Accuracy:{RF_hpt_accuracy} Precision:{RF_hpt_precison} Recall:{RF_hpt_recall} F1 score:{RF_hpt_f1},Best parameter:{RF_hpt_best_parameter}')

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END criterion=entropy, max_depth=3, n_estimators=200, oob_score=True; total time=  29.5s
[CV] END criterion=entropy, max_depth=3, n_estimators=200, oob_score=True; total time=  29.6s
[CV] END criterion=gini, max_depth=10, n_estimators=200, oob_score=True; total time=  48.0s
[CV] END criterion=gini, max_depth=10, n_estimators=200, oob_score=True; total time=  49.5s
[CV] END criterion=gini, max_depth=10, n_estimators=50, oob_score=True; total time=  12.6s
[CV] END criterion=gini, max_depth=10, n_estimators=50, oob_score=True; total time=  12.5s
[CV] END criterion=gini, max_depth=10, n_estimators=150, oob_score=True; total time=  37.0s
[CV] END criterion=gini, max_depth=10, n_estimators=150, oob_score=True; total time=  36.5s
[CV] END criterion=entropy, max_depth=None, n_estimators=200, oob_score=True; total time=  43.4s
[CV] END criterion=entropy, max_depth=None, n_estimators=200, oob_score=True; total time=  44.7s
[CV] EN

#XGBoost with default parameters

In [ ]:
XGB_model_dflt= XGBClassifier(random_state=7)

XGB_model_dflt.fit(xtrain,ytrain)

ypred=XGB_model_dflt.predict(xtest)

XGB_dflt_accuracy = accuracy_score(ytest, ypred)*100
XGB_dflt_precision = precision_score(ytest, ypred)*100
XGB_dflt_recall = recall_score(ytest, ypred)*100
XGB_dflt_f1 = f1_score(ytest, ypred)*100


print(f'Accuracy:{XGB_dflt_accuracy} Precision:{XGB_dflt_precision},Recall:{XGB_dflt_recall},F1_score:{XGB_dflt_f1}')

Accuracy:99.99065789641305 Precision:99.9974513418685,Recall:99.98386069229136,F1_score:99.99065555527238


#XGBoost with Hyper paramter tuning

In [ ]:
classifier_xgb=XGBClassifier()


## Hypeparameter Tuning for XGB
param_xgb= {
    'learning_rate': [0.01, 0.1, 0.2,0.3],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    #'subsample': [0.8, 1.0],
    #'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    }

xgb_model_hpt=RandomizedSearchCV(classifier_xgb,param_distributions=param_xgb,scoring='f1',cv=2,verbose=2)

xgb_hpt_ypred,xgb_hpt_accuracy,xgb_hpt_precison,xgb_hpt_recall,xgb_hpt_f1,xgb_hpt_best_parameter=evaluate_model(xgb_model_hpt,xtrain,ytrain,xtest,ytest)


print (f' Accuracy:{xgb_hpt_accuracy} Precision:{xgb_hpt_precison} Recall:{xgb_hpt_recall} F1 score:{xgb_hpt_f1},Best parameter:{xgb_hpt_best_parameter}')




Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END gamma=0.2, learning_rate=0.3, max_depth=7, n_estimators=100; total time=  11.5s
[CV] END gamma=0.2, learning_rate=0.3, max_depth=7, n_estimators=100; total time=   6.8s
[CV] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=300; total time=   8.9s
[CV] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=300; total time=   9.3s
[CV] END gamma=0, learning_rate=0.2, max_depth=5, n_estimators=200; total time=   5.2s
[CV] END gamma=0, learning_rate=0.2, max_depth=5, n_estimators=200; total time=   7.5s
[CV] END gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300; total time=   6.0s
[CV] END gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300; total time=   7.8s
[CV] END gamma=0.2, learning_rate=0.2, max_depth=7, n_estimators=200; total time=   3.7s
[CV] END gamma=0.2, learning_rate=0.2, max_depth=7, n_estimators=200; total time=   6.0s
[CV] END gamma=0.2, learning_rate=0.2, max_depth=5, n_estim

# CREATE TABLE TO COMPARE MODEL PERFORMANCE

## From out put we can observe all model has their accuracy precision recall and F1 score are nearly equal.

## Hence will create table to compare their F1 score only



In [ ]:
f1_scores = {
    'DT': {'Default': DT_dflt_f1, 'HyperTuned': DT_hpt_f1},
    'RF': {'Default': RF_dflt_f1, 'HyperTuned': RF_hpt_f1},
    'XGB': {'Default': XGB_dflt_f1, 'HyperTuned': xgb_hpt_f1}
}

# Create a DataFrame from the dictionary
F1_score_table = pd.DataFrame(f1_scores)

F1_score_table


,DT,RF,XGB
Default,99.985984,99.991080,99.990656
HyperTuned,99.984285,99.992354,99.989381


## From the F1 table we can see all three models are giving almost equal F1 score.

## We shall pick XGB with best parameters for  future prediction with user input


In [ ]:
xgb_hpt_best_parameter

{'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0}

In [ ]:
xgbmodel=XGBClassifier(**xgb_hpt_best_parameter)
xgbmodel.fit(xtrain,ytrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

#SAVNG THE MODEL

In [ ]:
import pickle
with open('capstoneClassification.pkl', 'wb') as f:
    pickle.dump(xgbmodel, f)
    pickle.dump(le, f)
    pickle.dump(feature_selected,f)

#Push this pickle file to same S3 bucket

In [ ]:
#Creating S3 object that will  push the file
push_to_s3= boto3.client('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)




In [ ]:
#Using object pushing/ uploading file to S3
push_to_s3.upload_file('/content/capstoneClassification.pkl',BUCKET_NAME, 'capstoneClassification.pkl')